In [1]:
import json
import sqlite3

co_key = "REPLACE ME"


In [29]:
import csv
import cohere
import json

co = cohere.Client('v4VoT1hN7tHnLDEs16Js1U7SQ453lA3SUP8Ww89k')  # This is your trial API key


def is_json(myjson):
    try:
        json.loads(myjson)
    except ValueError as e:
        return False
    return True


with sqlite3.connect('./db.sqlite') as con:
    cur = con.cursor()

    cur.execute("""
        SELECT count(DISTINCT requirementsDescription)
        from courses
        where requirementsDescription is not null
    """)
    num_distinct_requirements = cur.fetchone()[0]
    print("num_distinct_requirements: ", num_distinct_requirements)

for i in range(525, num_distinct_requirements, 5):
    cur.execute("""
        SELECT DISTINCT requirementsDescription
        FROM courses
        WHERE requirementsDescription is not null
        order by requirementsDescription
        LIMIT 5 OFFSET ?
    """, (i,))

    prereq_str = {}
    for ind, row in enumerate(cur.fetchall()):
        prereq_str[str(ind + 1)] = row[0]
    print(prereq_str)
    response = co.generate(
        model='command-nightly',
        prompt='Given a list of course requirements strings,  extract the requirements to JSON.\n\nIt should follow the format of:\n[id: string]: {\n  \"prereqs\": [string[]],\n  \"coreqs\": [string[]],\n  \"antireqs\": [string[]],\n  \"instructorPermission\": boolean,\n  \"requiredMajor\": string | null,\n  \"requiredTerm\": \"string | null,\n  \"otherNotes\": string\n}\n\n1A, 1B, 2A, 2B, etc. are different school terms.\nSlashes between course codes mean OR. For example, \"PHYS 123/453\" means \"PHYS 123\" OR \"PHYS 453\".\n\nOverall example:\n###\nINPUT: \n```\n{\n  1: \"Prereq: Level at least 4A Architecture.  Cumulative Major Average at least 73%; Coreq: ARCH 492\"\n  2: \"CHE Grad Students Only - all others require permission of department.\"\n  3: \"Pre/Co-req: ECE 650 or 750 Tpc 26, or instructor consent. Antireq: ECE 355, ECE 451, CS 445, CS 645, SE 463, ECE 452, CS 446, CS 646, SE 464\"\n  4: \"Prereq: Arts and Business students. Antireq: AFM 101, 102, BUS 127W/227W, 247W, MSCI 262\"\n  5: \"Prereq: (One of MATH 106, 114, 115, 136, 146, NE 112) and (One of MATH 128, 138, 148). Antireq: AMATH 251, 350, MATH 218, 228\"\n}\n```\n### \nOUTPUT: \n```\n{\n  1: {\n    \"prereqs\": [],\n    \"coreqs\": [\n      \"ARCH 492\"\n    ],\n    \"antireqs\": [],\n    \"instructorPermission\": false,\n    \"requiredMajor\": null,\n    \"minimumTerm\": \"4A\",\n    \"otherNotes\": \"Cumulative Major Average at least 73%\"\n  },\n  2: {\n    \"prereqs\": [],\n    \"coreqs\": [],\n    \"antireqs\": [\n    ],\n    \"instructorPermission\": false,\n    \"requiredMajor\": \"CHE Grad\",\n    \"minimumTerm\": null,\n    \"otherNotes\": \"\"\n  },\n  3: {\n    \"prereqs\": {\n      \"type\": \"OR\",\n      \"data\": [\n        \"ECE 650\",\n        \"ECE 750\"\n      ]\n    },\n    \"coreqs\": [],\n    \"antireqs\": [\n      \"ECE 355\",\n      \"ECE 451\",\n      \"CS 445\",\n      \"CS 645\",\n      \"SE 463\",\n      \"ECE 452\",\n      \"CS 446\",\n      \"CS 646\"\n    ],\n    \"instructorPermission\": true,\n    \"requiredMajor\": \"CHE Grad\",\n    \"minimumTerm\": null,\n    \"otherNotes\": \"\"\n  },\n  4: {\n    \"prereqs\": [],\n    \"coreqs\": [],\n    \"antireqs\": [\n      \"AFM 101\",\n      \"AFM 102\",\n      \"BUS 127W\",\n      \"BUS 227W\",\n      \"BUS 247W\",\n      \"MSCI 262\"\n    ],\n    \"instructorPermission\": false,\n    \"requiredMajor\": \"Arts and Business\",\n    \"minimumTerm\": null,\n    \"otherNotes\": \"\"\n  },\n  5: {\n    \"prereqs\": {\n      \"type\": \"AND\",\n      \"data\": [\n        {\n          \"type\": \"OR\",\n          \"data\": [\n            \"MATH 106\",\n            \"MATH 114\",\n            \"MATH 115\",\n            \"MATH 136\",\n            \"MATH 146\",\n            \"NE 112\"\n          ]\n        },\n        {\n          \"type\": \"OR\",\n          \"data\": [\n            \"MATH 128\",\n            \"MATH 138\",\n            \"MATH 148\"\n          ]\n        }\n      ]\n    },\n    \"coreqs\": [],\n    \"antireqs\": [\n      \"AMATH 251\",\n      \"AMATH 350\",\n      \"MATH 218\",\n      \"MATH 228\"\n    ],\n    \"instructorPermission\": false,\n    \"requiredMajor\": \"Arts and Business\",\n    \"minimumTerm\": null,\n    \"otherNotes\": \"\"\n  }\n}\n```\n###\nINPUT: \n```\n' + json.dumps(
            prereq_str) + '\n```\n###\nOUTPUT: \n',
        max_tokens=1878,
        temperature=0.9,
        k=0,
        stop_sequences=["###"],
        return_likelihoods='NONE')
    try:
        print(f"Predicted {i} of {num_distinct_requirements}")
        if is_json(response.generations[0].text):
            with open('parsed_requirements.tsv', 'at', newline='') as csvfile:
                writer = csv.writer(csvfile, delimiter='\t', lineterminator='\n')
                for key, value in json.loads(response.generations[0].text).items():
                    writer.writerow([prereq_str[key], value])
        else:
            print('Error: {}'.format(response.errors[0].message))
    except Exception as e:
        print(e)
        

num_distinct_requirements:  3799
{'1': 'Prereq: (MATH 135 with min. grade of 60% or MATH 145)&(MATH 128 with min. grade of 70% or (MATH 117 or 137) with min. grade of 60% or MATH 147)); Level at least 1B Hon Math or Math/Phys students only. Coreq: MATH 119 or 138 or 148. Antireq: STAT 220, 240', '2': 'Prereq: (MATH 135 with minimum grade of 80% or MATH 145) and (MATH 136 or 146) or instructor consent; Honours Mathematics students only. Antireq: CO 220, MATH 229, 239', '3': 'Prereq: (MATH 135 with minimum grade of 80% or MATH 145) and (MATH 136 or 146); Honours Mathematics students only. Antireq: CO 220, MATH 229, 239', '4': 'Prereq: (MATH 225/126 or 235 or 245) and (STAT 221 or 231 or 241). Antireq: ECON 321, STAT 331, 371, 373, 443', '5': 'Prereq: (MATH 235 or 245) and (AMATH/PMATH 331 or MATH 247 or PMATH 333). Antireq: PMATH 399 taken Winter 2019'}
Predicted 525 of 3799
{'1': 'Prereq: (MATH 235 or 245) and (MATH 237 or 247)', '2': 'Prereq: (MATH 235 or 245) and (STAT 231 with a grad